In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

# One-hot encoding of words or characters





A técnica de one-hot encoding consiste em associar um número inteiro para cada palavra e em seguida tornar este número em uma sequência de N-1 zeros e de 1 número um. Neste caso N é o tamanho do vocabulário. Esta técnica pode ser aplicada também aos caracteres de cada palavra ao invés das palavras em sí. A seguir apresentamos dois exemplos, um para palavras e outro para caracteres.

In [2]:
import numpy as np

# Estes são os nossos dados: duas amostras, uma entrada por amostra
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# Primeiro, criamos um índice para todos os tokens nos dados
token_index = {}
for sample in samples:
    # Neste exemplo as amostras foram tokenizadas com o método 'split'.
    for word in sample.split():
        if word not in token_index:
            # Designamos um índice único para cada palavra
            token_index[word] = len(token_index) + 1
            # Observe que o índice 0 não foi atribuido a nenhuma palavra

# Em seguida, vetorizamos nossas amostras.
# Serão utilizados apenas as primeiras `max_length` palavras de cada amostra.
max_length = 10

# Armazenamos os resultados em um numpy array:
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

results

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0

Exemplo com caracteres:

In [3]:
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  # All printable ASCII characters.
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.

results

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

Observe que o Keras possui utilitários para executar o one-hot encoding a partir de dados "raw" tanto com palavras quanto com caracteres. Estas funções executam várias tarefas importantes tais como extração de caracteres especiais de strings ou inclusão de apenas as N palavras mais comuns no seu conjunto de dados (uma restrição comum para evitar lidar com espaços vetoriais de entrada muito grandes). O exemplo a seguir executa o one-hot encoding de palavras através dos recursos da Keras:

In [0]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# Criamos um tokenizer, configurado para levar em consideração
# apenas as 1000 palavras mais comuns.
tokenizer = Tokenizer(num_words=1000)
# Esta etapa monte o índice de palavras
tokenizer.fit_on_texts(samples)

# Aqui tornamos strings em listas de índices inteiros.
sequences = tokenizer.texts_to_sequences(samples)

# É possível obter representações binárias do tipo one-hot diretamente.
# Outros métodos de vetorização também podem ser utilizados. 
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# Aqui recuperamos os índices de palavras que foram criados 
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


Uma variante da codificação one-hot é o chamado "truque de hash one-hot", que pode ser usado quando o número de tokens exclusivos no seu vocabulário é muito grande para ser explicitamente utilizado. Em vez de atribuir um índice a cada palavra e manter uma referência desses índices em um dicionário, pode-se usar um hash em vetores de tamanho fixo. Isso geralmente é feito com uma função de hash bem simples. A principal vantagem desse método é que ele acaba com a manutenção de um índice explícito de palavras, o que economiza memória e permite a codificação on-line dos dados (começando a gerar vetores de token imediatamente, antes de ver todos os dados disponíveis). A única desvantagem desse método é que ele é suscetível a "colisões de hash": duas palavras diferentes podem acabar com o mesmo hash e, subsequentemente, qualquer modelo de aprendizado de máquina que observe esses hashes não será capaz de diferenciar essas palavras. . A probabilidade de colisões de hash diminui quando a dimensionalidade do espaço de hash é muito maior que o número total de tokens únicos sendo hash. A seguir é apresentado um exemplo de codificação de palavras com a hashing trick.

In [4]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# As palavras serão armazenadas como vetores de tamanho 1.000
# Observe que se você tiver aproximadamente 1.000 palavras (ou mais)
# você começará a ter muitas "colisões" de hash, o que irá diminuir 
# a precisão deste método de codificação..
dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        # Codifica (hash) a palavra em um índice inteiro "aleatório" 
        # entre 0 e 1.000
        index = abs(hash(word)) % dimensionality
        results[i, j, index] = 1.

results

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])